# Interactive Broker Demo
### Please ensure IB TWS is running locally

In [98]:
import nest_asyncio
nest_asyncio.apply()

# Hide exception trace
import functools
ipython = get_ipython()
method_name = "showtraceback"
setattr(
    ipython,
    method_name,
    functools.partial(
        getattr(ipython, method_name),
        exception_only=True
    )
)

import ib_insync as ib

from src.config import *
from src.constant import *
from src.execution.marketdata import MarketData
from src.execution.interactive_broker import InteractiveBroker

## Connect to IB

In [108]:
PORT = 7496

In [109]:
self = InteractiveBroker(port=PORT)

In [110]:
self.connect()

[2021-01-29 16:02:02.066] [INFO] [interactive_broker:connect:29] IB Connected: <IB connected to 127.0.0.1:7496 clientId=204>


## Fetch portfolio greeks

### Fetch portfolio without marketdata

In [111]:
self.portfolio()

,symbol,sec_type,position,market_price,market_value,exp,strike,cp
0,TMF,STK,400.0,31.52,12608.00,,0.0,
1,TNA,OPT,4.0,2.14,857.97,20210205,67.0,P
2,TNA,OPT,-4.0,7.66,-3065.45,20210205,80.0,P


### Fetch portfolio with marketdata (market hours only)

In [112]:
df = self.portfolio(greeks=True)

Exception: No quote for TNA   210205P00067000. Please query it during market open.

In [44]:
df

,symbol,sec_type,position,market_price,market_value,bid,ask,mid,exp,strike,cp,delta,gamma,theta,vega,iv,und_price,timestamp
0,TMF,STK,400.0,31.76,12702.44,31.78,31.79,31.79,,0.0,,1.000,0.000,0.000,0.000,0.000,0.00,2021-01-28T12:47:11.509891-08:00
1,TNA,OPT,4.0,1.20,479.65,1.14,1.35,1.25,20210205,67.0,P,-0.144,0.016,-0.202,0.029,1.205,79.66,2021-01-28T12:47:11.514747-08:00
2,TNA,OPT,-4.0,4.95,-1978.03,4.90,5.15,5.03,20210205,80.0,P,-0.482,0.034,-0.296,0.047,1.002,79.66,2021-01-28T12:47:11.514747-08:00


### Fetch account summaries

In [ ]:
self.account_summary()

## Fetch Option Contracts
Note: this function can be very slow depending on the number of option contracts to be fetched.

### Stock Options

In [114]:
?self.options

Signature: self.options(symbol: str, percent_strike: float = 0.05, dte: int = 180) -> pandas.core.frame.DataFrame
Docstring:
Get a list of stock options with quotes for the given symbol.
:param symbol: stock symbol
:param percent_strike: (1 - percent_strike, 1 + percent_strike) * market_price
:param dte: maximum days to expiration from now (non-business days)

E.g. options('SPY', percent_strike=0.05, dte=180)

File:      ~/dev/crypto_bot/src/execution/interactive_broker.py
Type:      method


In [56]:
options = self.options('SPY')

[2021-01-28 12:57:58.756] [INFO] [marketdata:get_stock_options:90] Get 2160 potential option contracts.
[2021-01-28 12:59:33.575] [INFO] [marketdata:get_stock_options:93] Get 1506 option quotes.


In [62]:
options.head()

,symbol,bid,ask,mid,exp,strike,cp,delta,gamma,theta,vega,iv,und_price,timestamp
0,SPY,18.07,18.20,18.13,20210129,360.0,C,0.987,0.004,-0.038,0.008,0.421,377.97,2021-01-28T12:58:51.717928-08:00
1,SPY,17.05,17.20,17.12,20210129,361.0,C,0.987,0.004,-0.038,0.008,0.399,377.97,2021-01-28T12:58:50.741544-08:00
2,SPY,16.05,16.22,16.13,20210129,362.0,C,0.981,0.006,-0.055,0.008,0.398,377.97,2021-01-28T12:58:50.741544-08:00
3,SPY,15.57,15.71,15.64,20210129,362.5,C,0.980,0.006,-0.058,0.010,0.391,377.97,2021-01-28T12:58:50.723039-08:00
4,SPY,15.04,15.23,15.13,20210129,363.0,C,0.980,0.006,-0.055,0.010,0.377,377.97,2021-01-28T12:58:50.725463-08:00


### Future Options

#### Check available future contracts for a symbol

In [58]:
futures = self.futures('ES')

In [59]:
futures

,secType,conId,symbol,lastTradeDateOrContractMonth,strike,right,multiplier,exchange,primaryExchange,currency,localSymbol,tradingClass,includeExpired,secIdType,secId,comboLegsDescrip,comboLegs,deltaNeutralContract
0,FUT,396336017,ES,20210319,0.0,,50,GLOBEX,,USD,ESH1,ES,False,,,,[],None
1,CONTFUT,396336017,ES,20210319,0.0,,50,GLOBEX,,USD,ESH1,ES,False,,,,[],None
2,FUT,412889032,ES,20210618,0.0,,50,GLOBEX,,USD,ESM1,ES,False,,,,[],None
3,FUT,428520022,ES,20210917,0.0,,50,GLOBEX,,USD,ESU1,ES,False,,,,[],None
4,FUT,446091461,ES,20211217,0.0,,50,GLOBEX,,USD,ESZ1,ES,False,,,,[],None
5,FUT,461318816,ES,20220317,0.0,,50,GLOBEX,,USD,ESH2,ES,False,,,,[],None


#### Fetch options with given expiration date

In [60]:
fop = self.fop('ES', '20210319')

[2021-01-28 12:59:36.547] [INFO] [marketdata:get_future_options:138] ES: get 2280 potential options.
[2021-01-28 13:01:36.466] [INFO] [marketdata:get_future_options:141] ES: get 2163 option quotes.


In [61]:
fop.head()

,symbol,bid,ask,mid,exp,strike,cp,delta,gamma,theta,vega,iv,und_price,timestamp
0,ES,180.50,181.75,181.12,20210129,3595.0,C,0.989,0.000,-0.319,0.059,0.412,3775.5,2021-01-28T13:00:42.436289-08:00
1,ES,175.75,176.75,176.25,20210129,3600.0,C,0.988,0.000,-0.345,0.063,0.407,3775.5,2021-01-28T13:00:42.436289-08:00
2,ES,170.75,171.75,171.25,20210129,3605.0,C,0.987,0.000,-0.375,0.068,0.401,3775.5,2021-01-28T13:00:42.436289-08:00
3,ES,162.00,170.00,166.00,20210129,3610.0,C,0.986,0.000,-0.402,0.073,0.394,3775.5,2021-01-28T13:00:42.436289-08:00
4,ES,160.75,162.00,161.38,20210129,3615.0,C,0.984,0.001,-0.437,0.078,0.388,3775.5,2021-01-28T13:00:42.436289-08:00


## Fetch quote for Stock / Option / Future / Future Option

### Stock

In [81]:
self.quote('SPY')

{'symbol': 'SPY',
 'bid': 376.41,
 'ask': 376.48,
 'mid': 376.45,
 'exp': '',
 'strike': 0.0,
 'cp': '',
 'delta': 0.0,
 'gamma': 0.0,
 'theta': 0.0,
 'vega': 0.0,
 'iv': 0.0,
 'und_price': 0.0,
 'timestamp': '2021-01-28T15:10:53.752938-08:00'}

### Option

In [82]:
self.quote('SPY', expiry='20210129', strike=360.0, cp='C')

Exception: No quote for SPY   210129C00360000. Please query it during market open.

### Future

In [86]:
self.quote('ES', expiry='20210319')

{'symbol': 'ES',
 'bid': 3767.5,
 'ask': 3767.75,
 'mid': 3767.62,
 'exp': '20210319',
 'strike': 0.0,
 'cp': '',
 'delta': 0.0,
 'gamma': 0.0,
 'theta': 0.0,
 'vega': 0.0,
 'iv': 0.0,
 'und_price': 0.0,
 'timestamp': '2021-01-28T15:15:47.639021-08:00'}

### Future Option

In [84]:
self.quote('ES', expiry='20210129', strike=3595.0, cp='C', fop=True)

{'symbol': 'ES',
 'bid': 167.0,
 'ask': 181.75,
 'mid': 174.38,
 'exp': '20210129',
 'strike': 3595.0,
 'cp': 'C',
 'delta': 0.988,
 'gamma': 0.0,
 'theta': -0.328,
 'vega': 0.058,
 'iv': 0.42,
 'und_price': 3768.0,
 'timestamp': '2021-01-28T15:14:56.885247-08:00'}

## Disconnect IB

In [87]:
self.disconnect()

## Questions? @Ally on Slack